In [19]:
# from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy.ext.declarative import declarative_base
import sys
sys.path.append('../')
from sqlapp import SqlApp
import functools

In [37]:
s = SqlApp()

In [38]:
s.connect('postgresql://postgres@localhost:5532/sqlapp')

In [39]:
populate, create, Base = map(functools.partial(getattr, s), ['populate', 'create', 'Base'])

In [41]:
# /!\ DROP MODEL IN DB /!\
s.drop_models() 

## Many To One Relationship

In [42]:
"""
A basic Many to One Relationship
"""

class PlayerModel(Base):
    """Data model for players."""
    __tablename__ = "players"

    id = Column(Integer,
                primary_key=True,
                nullable=False)
    team_id = Column(Integer,
                     ForeignKey('teams.id'),
                     nullable=False)
    name = Column(String(100),
                  nullable=False)
    position = Column(String(100),
                      nullable=False)

    # Relationships
    team = relationship("TeamModel", backref="players")
    
    def __repr__(self):
        return '<Person model {}>'.format(self.id)


class TeamModel(Base):
    """Data model for people."""
    __tablename__ = "teams"

    id = Column(Integer,
                primary_key=True,
                nullable=False,
    )
    name = Column(String(100),
                  nullable=False)
    city = Column(String(100),
                  nullable=False)

    def __repr__(self):
        return '<Team model {}>'.format(self.id)

In [43]:
# PlayerModel.__table__.drop(engine)
# TeamModel.__table__.drop(engine)

# Many 2 Many Relationship

In [44]:


association_table = Table('association', Base.metadata,
    Column('left_id', Integer, ForeignKey('left.id')),
    Column('right_id', String, ForeignKey('right.id'))
)


class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    children = relationship(
        "Child",
        secondary=association_table,
        back_populates="parents")


class Child(Base):
    __tablename__ = 'right'
    id = Column(String, primary_key=True, )
    parents = relationship(
        "Parent",
        secondary=association_table,
        back_populates="children")

In [45]:
# /!\ WRITE MODEL IN DB /!\
# Base.metadata.create_all(engine)
s.create_models()

2020-02-05 00:13:09,491 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-05 00:13:09,494 INFO sqlalchemy.engine.base.Engine {'name': u'players'}
2020-02-05 00:13:09,645 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-05 00:13:09,648 INFO sqlalchemy.engine.base.Engine {'name': u'left'}
2020-02-05 00:13:09,663 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-05 00:13:09,665 INFO sqlalchemy.engine.base.Engine {'name': u'right'}
2020-02-05 00:13:09,672 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visib

In [46]:
TEAMS = [
    { 'name': 'PSG', 'city': 'Paris', 'id': 1},
    { 'name': 'OM', 'city': 'Marseille', 'id': 2},
    { 'name': 'OL', 'city': 'Lyon', 'id': 3},
    { 'name': 'LOSC', 'city': 'Lille', 'id': 4},
]
populate(TeamModel, TEAMS)

2020-02-05 00:13:12,890 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:12,893 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (id, name, city) VALUES (%(id)s, %(name)s, %(city)s)
2020-02-05 00:13:12,897 INFO sqlalchemy.engine.base.Engine ({'city': 'Paris', 'id': 1, 'name': 'PSG'}, {'city': 'Marseille', 'id': 2, 'name': 'OM'}, {'city': 'Lyon', 'id': 3, 'name': 'OL'}, {'city': 'Lille', 'id': 4, 'name': 'LOSC'})
2020-02-05 00:13:12,925 INFO sqlalchemy.engine.base.Engine COMMIT


In [47]:
PLAYERS = [
    {'name':'Mbappe', 'position': '9', 'team_id': '1'},
    {'name':'Neymar', 'position': '7', 'team_id': '1'},
    {'name':'Diawara', 'position': '8', 'team_id': '3'},
    {'name':'Benedeto', 'position': '9', 'team_id': '2'},
    {'name':'Payet', 'position': '7', 'team_id': '2'},
    {'name':'Pepe', 'position': '10', 'team_id': '4'},
]
populate(PlayerModel, PLAYERS)

2020-02-05 00:13:14,727 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:14,731 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,733 INFO sqlalchemy.engine.base.Engine {'position': '9', 'team_id': '1', 'name': 'Mbappe'}
2020-02-05 00:13:14,747 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,748 INFO sqlalchemy.engine.base.Engine {'position': '7', 'team_id': '1', 'name': 'Neymar'}
2020-02-05 00:13:14,757 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,762 INFO sqlalchemy.engine.base.Engine {'position': '8', 'team_id': '3', 'name': 'Diawara'}
2020-02-05 00:13:14,771 INFO sqlalchemy.engine.base.Engine INSERT INTO player

In [48]:
populate(Parent, [{'id':1, 'name': 'x'}, {'id':2, 'name': 'y'}])
populate(Child, [
    {'id': 'FR',}, {'id': 'ES'}, {'id': 'IT'},{'id': 'EN'}
])

2020-02-05 00:13:16,910 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:16,918 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id, name) VALUES (%(id)s, %(name)s)
2020-02-05 00:13:16,920 INFO sqlalchemy.engine.base.Engine ({'id': 1, 'name': 'x'}, {'id': 2, 'name': 'y'})
2020-02-05 00:13:16,933 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-05 00:13:16,957 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:16,961 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (%(id)s)
2020-02-05 00:13:16,964 INFO sqlalchemy.engine.base.Engine ({'id': 'FR'}, {'id': 'ES'}, {'id': 'IT'}, {'id': 'EN'})
2020-02-05 00:13:17,028 INFO sqlalchemy.engine.base.Engine COMMIT


In [49]:
p = s.session.query(Parent).filter_by(id=1).first()

2020-02-05 00:13:18,269 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:18,274 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id, "left".name AS left_name 
FROM "left" 
WHERE "left".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-05 00:13:18,276 INFO sqlalchemy.engine.base.Engine {'id_1': 1, 'param_1': 1}


In [50]:
p.children

2020-02-05 00:13:18,523 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right", association 
WHERE %(param_1)s = association.left_id AND "right".id = association.right_id
2020-02-05 00:13:18,525 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[]

In [51]:
p.children.append(s.session.query(Child).filter_by(id='ES').first())

2020-02-05 00:13:18,563 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-05 00:13:18,565 INFO sqlalchemy.engine.base.Engine {'id_1': 'ES', 'param_1': 1}


In [52]:
create(p)

2020-02-05 00:13:18,794 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id) VALUES (%(left_id)s, %(right_id)s)
2020-02-05 00:13:18,796 INFO sqlalchemy.engine.base.Engine {'left_id': 1, 'right_id': u'ES'}
2020-02-05 00:13:18,800 INFO sqlalchemy.engine.base.Engine COMMIT
